In [12]:
from enum import Enum
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
train_inputs = []
train_outputs = []
test_inputs = []
test_outputs = []

#Enum for all the categories and their column number
class Col(Enum):
    Gender = 1
    Menstrual = 2 
    SleepPrior = 3 
    Prescription = 4 
    OralAntibiotics = 5 
    TopAntibiotics = 6 
    Eyes = 7 
    Hands = 8 
    Race = 9 
    Feeling = 10 
    Symptoms = 11 
    Facial = 12 
    LastVomit = 13 
    EnvAllergies = 14 
    Snore = 15 
    GrindTeeth = 16 
    EarInfect = 17 
    AllergyTime = 18 
    Skin = 19 
    OralSurgery = 20 
    HeartDisease = 21 
    BadTeeth = 22 
    EatPrior = 23 
    DrinkPrior = 24 
    GumDay = 25 
    YogurtWeek = 26 
    TeaWeek = 27 
    CoffeeWeek = 28 
    SodaWeek = 29 
    FastFoodWeek = 30 
    EatOutWeek = 31 
    Straw = 32 
    AddedSugar = 33 
    Spicy = 34 
    ShareDrink = 35 
    MeatWeek = 36 
    FreshWeek = 37 
    BrushTongue = 38 
    BrushTeethWeek = 39 
    TobaccoWeek = 40 
    FlossWeek = 41 
    DentistYear = 42 
    Toothbrush = 43 
    Mouthwash = 44 
    Toothpaste = 45 
    WashHandsEat = 46 
    WashHandsRestroom = 47 
    ChangeToothbrush = 48 
    BiteNails = 49 
    OrthoDevice = 50 
    Device = 51 
    Stress = 52 
    HandleStress = 53 
    Organized = 54 
    RateStress = 55 
    AvgSleep = 56 
    AvgWork = 57 
    Setting = 58 
    Environment = 59 
    UpDown = 60 
    Weight = 61 
    ExerciseMonth = 62 
    Transportation = 63 
    Orientation = 64 
    NearDogsWeek = 65 
    NearCatsWeek = 66 
    ShareBathroom = 67 
    BooksYear = 68 
    Alarms = 69 
    Patience = 70 
    Lipstick = 71 
    IntroExtro = 72 
    Relgious = 73 
    OptiPessi = 74 
    SexuallyActive = 75 
    OralSex = 76 
    Music = 77 
    AvgGrade = 78 
    RateHappy = 79 
    Arts = 80 
    Kiss = 81 
    Roommates = 82 

    
#Enum for type of test
class Test(Enum):
    Ttest = 1
    Anova = 2
    

def sleepPrior(category, data):
    if category != Col.SleepPrior:
        return data
    
    #if SleepPrior, manipulate the data so that the subcategories don't go over 7
    #also create new subcategories to make it more general
    data[category.name] = data[category.name].replace(['0', '1', '2', '3'], 'less than 4')
    data[category.name] = data[category.name].replace(['4', '5', '6', '7'], '4 to 7')
    data[category.name] = data[category.name].replace(['8', '9', '10', '11', '12'], 'more than 7')
    return data


def prescription(category, data):
    if category != Col.Prescription:
        return data
    
    #manipulate the data so that if the prescription column has a specific prescription, replace with yes
    data[category.name] = data[category.name].replace(['None', 'none', 'no '], 'no')
    data.loc[(data[category.name] != 'no') & (data[category.name] != 'No') & (data[category.name] != 'nan'), 
             category.name] = 'yes'
    return data


def eyes(category, data):
    if category != Col.Eyes:
        return data
    
    #manipulate the data to make it more generic
    data[category.name] = data[category.name].replace('Green/blue/gold mix', 'hazel')
    return data


def perWeek(category, data):
    if category not in (Col.TeaWeek, Col.CoffeeWeek, Col.SodaWeek, Col.FastFoodWeek, Col.BrushTeethWeek, 
                        Col.FlossWeek, Col.BooksYear, Col.GumDay, Col.EatOutWeek, Col.AvgSleep, Col.AvgWork, 
                        Col.ExerciseMonth) :
        return data
    
    #manipulate the data so results don't show as dates
    data[category.name] = data[category.name].replace('5-Jan', '1-5')
    data[category.name] = data[category.name].replace('10-Jun', '6-10')
    data[category.name] = data[category.name].replace('15-Nov', '11-15')
    
    data[category.name] = data[category.name].replace('2-Jan', '1-2')
    data[category.name] = data[category.name].replace('4-Mar', '3-4')
    data[category.name] = data[category.name].replace('6-May', '5-6')
    data[category.name] = data[category.name].replace('8-Jul', '7-8')
    data[category.name] = data[category.name].replace('10-Sep', '9-10')
    data[category.name] = data[category.name].replace('12-Nov', '11-12')
    
    if category == Col.FlossWeek:
        data[category.name] = data[category.name].replace('5-Nov', '11-15')
    
    return data


def numToWords(category, data):
    if category not in (Col.AddedSugar, Col.Straw, Col.Lipstick):
        return data
    
    #manipulate the data so results don't show as dates
    data[category.name] = data[category.name].replace('1', 'never')
    data[category.name] = data[category.name].replace('2', 'rarely')
    data[category.name] = data[category.name].replace('3', 'sometimes')
    data[category.name] = data[category.name].replace('4', 'often')
    data[category.name] = data[category.name].replace('5', 'always')
    
    return data


def mouthwash(category, data):
    if category != Col.Mouthwash:
        return data
    
    #manipulate the data so that results are in form of yes or no
    data.loc[(data[category.name] != 'never') & (data[category.name] != 'No') & (data[category.name] != 'nan'), 
             category.name] = 'yes'
    data[category.name] = data[category.name].replace('never', 'no')
    return data


def religious(category, data):
    if category != Col.Relgious:
        return data
    
    #manipulate the data so that results are in form of yes or no
    data.loc[(data[category.name] != 'No') & (data[category.name] != 'nan'), category.name] = 'yes'
    data[category.name] = data[category.name].replace('never', 'no')
    return data


def orientation(category, data):
    if category != Col.Orientation:
        return data
    
    #fix spelling mistakes, change bicurious to bisexual to generalize data
    data[category.name] = data[category.name].replace('hetrosexual', 'heterosexual')
    data[category.name] = data[category.name].replace(['bicurious', 'Bicurious'], 'bisexual')
    return data
    
    
def runTests(data, category, outputMap, showAllClasses):
    subcategory = []
    categoryDF = []
    bactClasses = []
    sigClasses = []
    inputs = []
    
    #decided not to do these groups
    if category.name in ('Menstrual', 'Race', 'Feeling', 'Symptoms', 'EnvAllergies', 'Skin', 'EatPrior',
                        'DrinkPrior', 'TobaccoWeek', 'Toothpaste', 'BiteNails', 'Transportation', 'Environment', 
                        'NearDogsWeek', 'NearCatsWeek'):
        return False
    
    
    # fix the inconsistant data
    data = sleepPrior(category, data)
    data = prescription(category, data)
    data = eyes(category, data)
    data = perWeek(category, data)
    data = numToWords(category, data)
    data = mouthwash(category, data)
    data = religious(category, data)
    data = orientation(category, data)

    #change the category into its own separate dataframe

    #get list of subcategories
    try:
        data[category.name] = data[category.name].str.lower()
    except:
        data[category.name]
    subcategory = data[category.name].drop_duplicates().values.tolist()
    
    #remove nans (where people didn't put in an answer)
    try:
        subcategory = [subcat.lower().strip() for subcat in subcategory if str(subcat) != 'nan' 
                   and str(subcat) != 'don\'t know' and str(subcat) != 'do not know' and str(subcat) != 'don?t know']
    except:
        subcategory = [subcat for subcat in subcategory if str(subcat) != 'nan' 
                   and str(subcat) != 'don\'t know' and str(subcat) != 'do not know' and str(subcat) != 'don?t know']
        
    #remove any more duplicates
    subcategory = list(set(subcategory))

    #remove outliers/format the data for specific categories
    if category == Col.Spicy:
        subcategory.remove('sometimes')
    if category == Col.Hands:
        subcategory.remove('ambidextrous')
    if category == Col.BrushTeethWeek:
        subcategory.remove('42689');
    

    #decide on which test to use
    if len(subcategory) == 2:
        test = Test.Ttest
    else:
        test = Test.Anova
        
    #load dataframes for each subcategory of all bacterial classes\n",
    for x in range(0, len(subcategory)):
        categoryDF.append(data[data[category.name] == subcategory[x]].iloc[:, 83:125])
        
    print(category.name)
    print(subcategory)

    #perform t-test if applicable
    if test == Test.Ttest:
        
        #go through all bacterial classes to perform t-tests on them
        for x in range(0, 42):
            ttest = ttest_ind(categoryDF[0].iloc[:, x],categoryDF[1].iloc[:, x])
            pvalue = ttest[1]
            
            #keep a list of all bacteria classes and their index
            if len(bactClasses) != 42:
                bactClasses.append(categoryDF[0].iloc[:,x].name)

            #print classes that are significant
            if pvalue <= 0.05 or showAllClasses:
                #keep track of significant classes
                sigClasses.append(categoryDF[0].iloc[:,x].name)
                
                print(categoryDF[0].iloc[:,x].name, "\nP-value: {:4.6f}".format(pvalue))
        print('\n')
        
    #perform anova test is applicable
    if test == Test.Anova:
        
        #go through all bacterial classes to perform anova tests on them
        for x in range(0,42):
            
            #keep track of how many subcategories there are to put into anova test
            arg = []
            for y in range(0, len(subcategory)):
                arg.append(categoryDF[y].iloc[:, x])
            anova = stats.f_oneway(*(a for a in arg))
            pvalue = anova[1]
            
            #keep a list of all bacteria classes and their index
            if len(bactClasses) != 42:
                bactClasses.append(categoryDF[0].iloc[:,x].name)
            
            #print classes that are significant
            if pvalue <= 0.05 or showAllClasses:
                #keep track of significant classes
                sigClasses.append(categoryDF[0].iloc[:,x].name)
                
                print(categoryDF[0].iloc[:,x].name, "\nP-value: {:4.6f}".format(pvalue))
        print('\n')
        
    #get and set inputs, key = bactClass, value = list of all values under that class - Design 1
#     for c in sigClasses:
#         classValue = []
#         for index, row in data.iterrows():
#             if row[category.name] in subcategory:
#                 classValue.append(row[c])
#         inputs[c] = classValue
#     print(inputs)

    #get and set inputs Design 2
    for index, row in data.iterrows():
        classValue = []
        for c in sigClasses:
            if row[category.name] in subcategory:
                classValue.append(row[c])
        if row[category.name] in subcategory:
            inputs.append(classValue)
    print(len(inputs), inputs)
    
    #add subcategories to outputMap
    outputMap[category.name] = subcategory
    print(outputMap)
    
    #get and set outputs so that it looks like [a,b,c]
    outputs = []
    for index, row in data.iterrows():
        if row[category.name] in subcategory:
            outputs.append(row[category.name])
    print(len(outputs), outputs)
    
    #replace string outputs with numbers
    if outputs and isinstance(outputs[0], str):
        for i in range(0, len(outputs)):
            outputs[i] = subcategory.index(outputs[i])
    print(len(outputs), outputs)
    
    #separate training and testing data
    for i in range(0, len(inputs)):
        if i % 2 == 1:
            train_inputs.append(inputs[i])
            train_outputs.append(outputs[i])
        else:
            test_inputs.append(inputs[i])
            test_outputs.append(outputs[i])  
    
    

class Main():
    data = pd.read_csv('MasterSheet.csv')
    outputMap = {}
    showAllClasses = False
    
    runTests(data, Col.Gender, outputMap, showAllClasses)
    
#     for x in range(1, 82):
#         runTests(data, Col(x), outputMap, showAllClasses)



Gender
['male', 'female']
Bacilli 
P-value: 0.040059
Fusobacteria 
P-value: 0.000003
Epsilonproteobacteria 
P-value: 0.021050
Alphaproteobacteria 
P-value: 0.038678
Thermotogae 
P-value: 0.012158


106 [[3.9442973119999998, 0.076904877, -0.33510018199999997, -0.342277971, -0.353762432], [3.9723758339999997, 0.190906131, -0.362045826, -0.357471775, -0.381358487], [3.760585735, 0.602311415, -0.370152097, -0.361497044, -0.376256186], [3.114230698, -0.22210792399999998, -0.339227998, -0.318056067, -0.331408263], [5.343307545, 0.134594481, -0.308472788, -0.30015785300000003, -0.319163419], [3.503269346, 0.480379028, -0.363317868, -0.366886806, -0.377593619], [3.746173966, -0.021308512, -0.35812048399999996, -0.329250886, -0.343685685], [2.187348146, -0.207054766, -0.33773129700000004, -0.328490128, -0.336431133], [3.77699665, 0.203021108, -0.31253190000000003, -0.35530704, -0.35530704], [4.293834467, -0.064573619, -0.349397739, -0.333604432, -0.351576126], [4.398598967, 0.094596186, -0.3224

In [27]:
# following https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/_index.ipynb
# as a tutorial

import tensorflow as tf
from tensorflow.keras import layers

mnist = tf.keras.datasets.mnist

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0


# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(512, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(10, activation=tf.nn.softmax)
# ])
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(np.array(train_inputs), np.array(train_outputs), epochs=1000)

model.evaluate(np.array(test_inputs), np.array(test_outputs))

Epoch 1/1000
53/53 [==============================] - 1s 18ms/step - loss: 1.8146 - acc: 0.5472
Epoch 2/1000
53/53 [==============================] - 0s 108us/step - loss: 1.6988 - acc: 0.5849
Epoch 3/1000
53/53 [==============================] - 0s 94us/step - loss: 1.5824 - acc: 0.6038
Epoch 4/1000
53/53 [==============================] - 0s 107us/step - loss: 1.4695 - acc: 0.6038
Epoch 5/1000
53/53 [==============================] - 0s 169us/step - loss: 1.3632 - acc: 0.6038
Epoch 6/1000
53/53 [==============================] - 0s 113us/step - loss: 1.2659 - acc: 0.6226
Epoch 7/1000
53/53 [==============================] - 0s 132us/step - loss: 1.1741 - acc: 0.6415
Epoch 8/1000
53/53 [==============================] - 0s 132us/step - loss: 1.0894 - acc: 0.6604
Epoch 9/1000
53/53 [==============================] - 0s 151us/step - loss: 1.0181 - acc: 0.6981
Epoch 10/1000
53/53 [==============================] - 0s 188us/step - loss: 0.9526 - acc: 0.6981
Epoch 11/1000
53/53 [==========

Epoch 84/1000
53/53 [==============================] - 0s 151us/step - loss: 0.5345 - acc: 0.6792
Epoch 85/1000
53/53 [==============================] - 0s 151us/step - loss: 0.5340 - acc: 0.6792
Epoch 86/1000
53/53 [==============================] - 0s 113us/step - loss: 0.5331 - acc: 0.6792
Epoch 87/1000
53/53 [==============================] - ETA: 0s - loss: 0.5073 - acc: 0.718 - 0s 188us/step - loss: 0.5366 - acc: 0.6792
Epoch 88/1000
53/53 [==============================] - 0s 113us/step - loss: 0.5313 - acc: 0.6792
Epoch 89/1000
53/53 [==============================] - 0s 94us/step - loss: 0.5330 - acc: 0.6792
Epoch 90/1000
53/53 [==============================] - 0s 113us/step - loss: 0.5302 - acc: 0.6792
Epoch 91/1000
53/53 [==============================] - 0s 113us/step - loss: 0.5295 - acc: 0.6792
Epoch 92/1000
53/53 [==============================] - 0s 113us/step - loss: 0.5297 - acc: 0.6792
Epoch 93/1000
53/53 [==============================] - 0s 113us/step - loss: 0.53

53/53 [==============================] - 0s 131us/step - loss: 0.5004 - acc: 0.6792
Epoch 167/1000
53/53 [==============================] - 0s 169us/step - loss: 0.4965 - acc: 0.6792
Epoch 168/1000
53/53 [==============================] - 0s 118us/step - loss: 0.4961 - acc: 0.7170
Epoch 169/1000
53/53 [==============================] - 0s 94us/step - loss: 0.4954 - acc: 0.7170
Epoch 170/1000
53/53 [==============================] - 0s 129us/step - loss: 0.4921 - acc: 0.7170
Epoch 171/1000
53/53 [==============================] - 0s 113us/step - loss: 0.4916 - acc: 0.6981
Epoch 172/1000
53/53 [==============================] - 0s 113us/step - loss: 0.4924 - acc: 0.6981
Epoch 173/1000
53/53 [==============================] - 0s 263us/step - loss: 0.4916 - acc: 0.7170
Epoch 174/1000
53/53 [==============================] - 0s 107us/step - loss: 0.4913 - acc: 0.6981
Epoch 175/1000
53/53 [==============================] - 0s 113us/step - loss: 0.4894 - acc: 0.6981
Epoch 176/1000
53/53 [====

53/53 [==============================] - 0s 150us/step - loss: 0.4615 - acc: 0.7547
Epoch 248/1000
53/53 [==============================] - 0s 159us/step - loss: 0.4612 - acc: 0.7547
Epoch 249/1000
53/53 [==============================] - 0s 103us/step - loss: 0.4600 - acc: 0.7547
Epoch 250/1000
53/53 [==============================] - 0s 133us/step - loss: 0.4598 - acc: 0.7547
Epoch 251/1000
53/53 [==============================] - 0s 110us/step - loss: 0.4644 - acc: 0.7170
Epoch 252/1000
53/53 [==============================] - 0s 103us/step - loss: 0.4659 - acc: 0.6981
Epoch 253/1000
53/53 [==============================] - 0s 132us/step - loss: 0.4559 - acc: 0.7547
Epoch 254/1000
53/53 [==============================] - 0s 131us/step - loss: 0.4579 - acc: 0.7358
Epoch 255/1000
53/53 [==============================] - ETA: 0s - loss: 0.5068 - acc: 0.687 - 0s 113us/step - loss: 0.4744 - acc: 0.6981
Epoch 256/1000
53/53 [==============================] - 0s 136us/step - loss: 0.4690 -

53/53 [==============================] - 0s 132us/step - loss: 0.4423 - acc: 0.7736
Epoch 329/1000
53/53 [==============================] - 0s 119us/step - loss: 0.4392 - acc: 0.7925
Epoch 330/1000
53/53 [==============================] - 0s 144us/step - loss: 0.4310 - acc: 0.7358
Epoch 331/1000
53/53 [==============================] - 0s 169us/step - loss: 0.4333 - acc: 0.7547
Epoch 332/1000
53/53 [==============================] - 0s 188us/step - loss: 0.4380 - acc: 0.7170
Epoch 333/1000
53/53 [==============================] - ETA: 0s - loss: 0.3918 - acc: 0.812 - 0s 320us/step - loss: 0.4413 - acc: 0.7358
Epoch 334/1000
53/53 [==============================] - 0s 148us/step - loss: 0.4319 - acc: 0.7736
Epoch 335/1000
53/53 [==============================] - 0s 117us/step - loss: 0.4334 - acc: 0.7547
Epoch 336/1000
53/53 [==============================] - 0s 395us/step - loss: 0.4345 - acc: 0.7547
Epoch 337/1000
53/53 [==============================] - 0s 169us/step - loss: 0.4314 -

...

53/53 [==============================] - 0s 132us/step - loss: 0.3312 - acc: 0.8491
Epoch 736/1000
53/53 [==============================] - 0s 126us/step - loss: 0.3287 - acc: 0.8302
Epoch 737/1000
53/53 [==============================] - 0s 94us/step - loss: 0.3343 - acc: 0.8491
Epoch 738/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3379 - acc: 0.8679
Epoch 739/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3273 - acc: 0.8491
Epoch 740/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3363 - acc: 0.8491
Epoch 741/1000
53/53 [==============================] - ETA: 0s - loss: 0.3263 - acc: 0.875 - 0s 114us/step - loss: 0.3425 - acc: 0.8491
Epoch 742/1000
53/53 [==============================] - 0s 132us/step - loss: 0.3332 - acc: 0.8302
Epoch 743/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3306 - acc: 0.8491
Epoch 744/1000
53/53 [==============================] - 0s 140us/step - loss: 0.3395 - 

53/53 [==============================] - 0s 113us/step - loss: 0.3174 - acc: 0.8679
Epoch 817/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3156 - acc: 0.8679
Epoch 818/1000
53/53 [==============================] - 0s 112us/step - loss: 0.3193 - acc: 0.8679
Epoch 819/1000
53/53 [==============================] - 0s 147us/step - loss: 0.3155 - acc: 0.8679
Epoch 820/1000
53/53 [==============================] - ETA: 0s - loss: 0.3784 - acc: 0.843 - 0s 113us/step - loss: 0.3160 - acc: 0.8679
Epoch 821/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3159 - acc: 0.8679
Epoch 822/1000
53/53 [==============================] - 0s 109us/step - loss: 0.3220 - acc: 0.8491
Epoch 823/1000
53/53 [==============================] - 0s 130us/step - loss: 0.3236 - acc: 0.8302
Epoch 824/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3170 - acc: 0.8491
Epoch 825/1000
53/53 [==============================] - 0s 132us/step - loss: 0.3158 -

53/53 [==============================] - 0s 94us/step - loss: 0.3016 - acc: 0.8679
Epoch 897/1000
53/53 [==============================] - 0s 94us/step - loss: 0.3029 - acc: 0.8491
Epoch 898/1000
53/53 [==============================] - 0s 97us/step - loss: 0.3054 - acc: 0.8302
Epoch 899/1000
53/53 [==============================] - 0s 93us/step - loss: 0.3116 - acc: 0.8491
Epoch 900/1000
53/53 [==============================] - 0s 132us/step - loss: 0.3055 - acc: 0.8491
Epoch 901/1000
53/53 [==============================] - 0s 119us/step - loss: 0.3016 - acc: 0.8491
Epoch 902/1000
53/53 [==============================] - 0s 128us/step - loss: 0.3050 - acc: 0.8679
Epoch 903/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3033 - acc: 0.8491
Epoch 904/1000
53/53 [==============================] - 0s 95us/step - loss: 0.3045 - acc: 0.8491
Epoch 905/1000
53/53 [==============================] - 0s 113us/step - loss: 0.3001 - acc: 0.8491
Epoch 906/1000
53/53 [========

Epoch 978/1000
53/53 [==============================] - 0s 94us/step - loss: 0.2964 - acc: 0.8491
Epoch 979/1000
53/53 [==============================] - 0s 113us/step - loss: 0.2980 - acc: 0.8491
Epoch 980/1000
53/53 [==============================] - 0s 92us/step - loss: 0.2919 - acc: 0.8491
Epoch 981/1000
53/53 [==============================] - 0s 113us/step - loss: 0.2913 - acc: 0.8491
Epoch 982/1000
53/53 [==============================] - 0s 132us/step - loss: 0.2910 - acc: 0.8491
Epoch 983/1000
53/53 [==============================] - 0s 113us/step - loss: 0.2918 - acc: 0.8491
Epoch 984/1000
53/53 [==============================] - 0s 113us/step - loss: 0.2915 - acc: 0.8491
Epoch 985/1000
53/53 [==============================] - 0s 113us/step - loss: 0.2912 - acc: 0.8491
Epoch 986/1000
53/53 [==============================] - 0s 116us/step - loss: 0.2917 - acc: 0.8491
Epoch 987/1000
53/53 [==============================] - 0s 126us/step - loss: 0.2903 - acc: 0.8491
Epoch 988/10

[1.1318156842915517, 0.6226415128078101]